In [35]:
%run ./FIR_Helper_Methods.ipynb
#Import the helper nethod and libraries

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

# To Do:

3. http://mithra.eecs.umich.edu/demo/ranking/
4. Split into multiple files
5. Apply IBM 360 and Aequitas
7. implement this check for demographic parity.... https://www.prevuehr.com/resources/insights/adverse-impact-analysis-four-fifths-rule/

* Add abitity to select the protected attributes to analysis (race/sex/race with sex, etc)
* Add abitity to change the order of the protected attributes for the analysis
* https://towardsdatascience.com/an-easier-way-to-encode-categorical-features-d840ff6b3900 other encoding types.
* add the % value to the bar chart in case where there is only 1 protected attribute
* add a bubble chart 


****Pairwise Ranking vs Pairwise Comparison (individual fairness)


Pandas CrossTab https://pbpython.com/pandas-crosstab.html
Pandas Styles https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html

///
Use this as part of the class to print statistics on any dropdown. 
print ("Mimimum FYA:", dp.get_df()['ZFYA'].max())
print ("Maximum FYA:", dp.get_df()['ZFYA'].min())
print ("Maximum FYA:", dp.get_df()['ZFYA'].mean())
Type
Missing Values
Number of unique entries
Other relevant Data>
/////
https://stackoverflow.com/questions/45286696/how-to-display-dataframe-next-to-plot-in-jupyter-notebook/45287049


TODO CODE:
display (dpUI.df.head())
sns.pairplot(df.sample(1000))#Think about only showing a sample for the pairplot!
sns_plot = sns.pairplot(df, size=2.0)
sns_plot.savefig("pairplot.png")#Tsee if speeds up by saving it to file
plt.clf() # Clean parirplot figure from sns 
Image(filename='pairplot.png') # Show pairplot as image

# About the Law School Admission data
The Law School Admission Council conducted a survey across 163 law schools in the United States

https://www.luciusbynum.com/project/counterfactual-fairness/

"Predicting performance from LSAT and GPA"
https://poseidon01.ssrn.com/delivery.php?ID=826070013069091113081012005005126026118004050083020091078110083112125066089115084102096048123000045063041004069096102021085026019076007034050100003116079070007082072053079042111014069095100022002098069072117086064008106018026097120007022065003115006122&EXT=pdf


It contains information on 21,790 law students such as:
- Entrance exam scores (LSAT)
- Grade-point average (GPA) collected prior to law school,
- sander_index' - Sander proposes combining the LSAT and undergraduate GPA into a single
weighted average, using weights that correspond roughly to those used in many law schools. We refer to this average as the “Sander Index.” Because the units of this index are difficult to interpret, we convert it to a percentile score. This score ranges from 0 to 100, and represents the percentage of law school matriculants with lower index scores. That is, a student with a percentile score of 75 has better academic credentials than three quarters of law school matriculants, but worse credentials than the remaining quarter. see http://public.econ.duke.edu/~hf14/teaching/povertydisc/readings/rothstein_yoon_2007.pdf

- first_pf - Whether or not the BAR exam was passed or failed
- region_first
- First year average grade (FYA) - Label used for training purposes. This is an Z score
Z-Scores are raw scores expressed in standard deviation units, relative to the mean score. Positive Z‐
scores indicate a raw score that is above the mean, negative Z‐scores indicate a raw score that is below
the mean, and a Z‐score of zero indicates a raw score that is equal to the mean. In a normally‐
distributed set of data, the general rule states that 68% of all scores will fall within ±1 SD of the mean;
95% of all scores will fall within ±2 SD, and 99.7% of all scores within ±3 SD. Z‐scores between ‐2.00 and
+2.00 are therefore considered relatively ordinary, while values greater than ‐2.00 and +2.00 are
unusual.

                      
In this scenario we will use the data to predict the success of prospective students in their first year(ZFYA) of law school. We can imagine such a model being used by an admissions department as part of deciding who to accept into their program. The school would also like to make sure these predictions are not biased by an individual’s race and sex. However, the LSAT, GPA, and FYA scores, may be biased due to social factors. 

we generate predictors Y for a logistic regression model, ranking the output. 

# About the Compas data

# About the Loan approval Data

https://datahack.analyticsvidhya.com/contest/practice-problem-loan-prediction-iii/

http://bayesian-intelligence.com/publications/TR2010_1_zonneveldt_korb_nicholson_bn_class_credit_data.pdf

    

# Pre-process the data
Machine learning algorithms require numerical inputs. Our features will come in various forms. Some will be continuous, others categorical (in numeric or text format). Machine learning algorithms cannot work with variables in text form, we must perform certain preprocessing steps to get our data in the right format.


- Identify the protected attributes and the label (y) in the dataset so that these may be split  out when necessary
- Remove any obviously unnecessary columns
- Review any correlations and remove any obviously correlated input columns
- Standardise/normalise or scale any necessary numerical features
- Review protected attributes and
    * Consolidate values for better representation e.g Race => White and Non-White
    * Rename any necessary category/Text values (Label Encoding)
    * Change column data types of category/text data to 'int' (Label Encoding)
- one-hot encode the categorical features.
- Decide if the model will be trained with or without the protected attributes






In [3]:
class data_pre_process_UI():

    def __init__(self):
        self.helper_methods = helper_methods()
        pd.set_option('display.max_colwidth', -1)
        self.df = pd.DataFrame() # Original Dataframe uploaded
        self.df_url = None
        self.y_label_df = pd.DataFrame() # Dataframe containing label(score) and also the range(upper or lower)
        self.non_protected_feature_set_df = pd.DataFrame() # Dataframe containing features except protected
        self.protected_feature_set_df = pd.DataFrame() # Dataframe containing features except protected

        self.text_color = "#33C1FF"
        # A dictionary, mapping feature encoding values
        self.encoding_dict = {}
        self.HIGH_RANGE_POSITIVE = True
        self.Y_BINARY = False
        self.Y_CONTINUOUS = False
        self.y_min_rank = 0
        self.y_mid_rank = 0.5
        self.y_max_rank = 1
        self.renamed = {}
        self.referenceNames = {}
        self.proxyAttributes = []
        self.MLAttributes = []
        

        
        self.style = {'description_width': 'initial'}
        self.layout = {'width': 'auto'}
        self.layout_short = {'width': 'auto'}
        
        self.auto_width_layout = {'width': 'auto'}     # override the default width of the button to 'auto' to let the button grow

        self.fileUploader = FileUpload(accept='.csv',
                                       multiple=False,
                                       disabled=False,
                                       style_button='color: darkblue; background-color: lightsalmon; width: 180px;',
                                       compress_level=9
                                       )
        

        self.missingDataRowsThresholdSlider = widgets.IntSlider(
                                value=70,
                                description= "Rows threshold(percent)" ,
                                max=100,
                                min=0,
                                style=self.style,
                                layout = self.auto_width_layout)
        
        
        self.missingDataColumnsThresholdSlider = widgets.IntSlider(
                                value=70,
                                description= "Columns threshold(percent)" ,
                                max=100,
                                min=0,
                                style=self.style,
                                layout = self.auto_width_layout)
        

        
        
        self.remove_rows_cols_button = widgets.Button(
            description='Remove',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Remove rows or columns where x% of data is missing',
            icon='',
            layout = self.auto_width_layout
        )
        
         
            
        self.selectColumnWithMissingData = widgets.Dropdown(
            options = ["--select--"],
            value = "--select--",
            description='Column with missing data:',
            disabled=False,
            style=self.style,   
            layout = self.auto_width_layout
        )
        
        
        self.selectImputeMethod = widgets.Dropdown(
            description='Replace missing values with:',
            disabled=False,
            style=self.style, 
            layout = self.auto_width_layout
        )
        
        
        self.apply_imputation_button = widgets.Button(
            description='Apply imputation',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Apply  imputation to the column',
            icon='',
            layout = self.auto_width_layout
        )
        
        
        self.selectYFeature = widgets.Dropdown(
            options=  self.df.columns.values,
            value=None,
            description='Output Label(y):',
            disabled=False,
            style=self.style,
            layout = self.auto_width_layout
        )
        

        self.selectXFeatures = widgets.SelectMultiple(
            options = self.df.columns.values,
            description='Features',
            disabled=False,
            style=self.style,
            layout = self.auto_width_layout
        )
        

        self.selectProtectedAttributes = widgets.SelectMultiple(
            options=self.df.columns.values,
            description='Protected Attribute(s)',
            disabled=False,
            style=self.style,
            layout = self.auto_width_layout
        )
        
        self.theSelectedInputFeaturesChoice = widgets.SelectMultiple(
            options = [],
            description='Features',
            disabled=False,
            style=self.style,
            layout = self.auto_width_layout
        )
        
        self.theSelectedInputFeaturesDropDown = widgets.Dropdown(
            options = [],
            description='Features',
            disabled=False,
            style=self.style,
            layout = self.auto_width_layout
        )
    

        self.selectImpact = widgets.RadioButtons(
            options={'A more positive impact on an individual': 1,
                     'A more negative impact on an individual': 0},
            description='A prediction in the high range will have:',
            disabled=False,
            style=self.style,
            layout = self.auto_width_layout
        )
        

        self.newNameForValue = widgets.Text(
            description='New Value Name',
            disabled=False,
            tooltip='enter the new value name',
            style = self.style,
            layout = self.auto_width_layout
        )
        

        self.choosenFeatureValues = widgets.SelectMultiple(
            description='Select value(s)',
            disabled=False,
            style=self.style,
            layout = self.auto_width_layout
        )


        self.merge_values_button = widgets.Button(
            description='Merge',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Merge feature values',
            icon='',
            layout = self.auto_width_layout
        )

        self.descriptionText = widgets.Text(
            description='Description of value',
            disabled=False,
            tooltip='Save a description for the value e.g "protected" or the original name',
            style = self.style,
            layout = self.auto_width_layout
        )

        self.save_description_button = widgets.Button(
            description='Save',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            icon='check',
            layout = self.auto_width_layout
        )


        self.label_encode_button = widgets.Button(
            description='Encode',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='click to apply label encoding',
            layout = self.auto_width_layout
        )
        

        self.one_hot_encode_button = widgets.Button(
            description='Encode',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='click to apply one-hot-encoding',
            layout = self.auto_width_layout
            
        )

        
        self.change_column_type_button = widgets.Button(
            description='Change',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            icon='check',
            layout = self.auto_width_layout
        )
        

        self.selectProtected = widgets.ToggleButtons(
            options=['Yes', 'No'],
            value='Yes',
            description='Select:',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltips=['Tooltip1', 'Tooltip2'],
            style=self.style,
            layout = self.auto_width_layout
        )

        
        self.selectColumnType = widgets.Dropdown(
            description='Change Column data type',
            options=['str', 'int'],
            value=None,
            disabled=False,
            style=self.style,
            layout = self.auto_width_layout
        )
        
        
        self.selectScaleType = widgets.Dropdown(
            description='Method to apply',
            options ={"--Select--":"SELECT",
                      "View Standard Scalar":"STANDARD_SCALAR",
                      "View Min Max Scalar": "MIN_MAX_SCALAR",
                      "View Robust Scalar":"ROBUST_SCALAR",
                      "View Normalise": "NORMALIZER"},
            value="SELECT",
            disabled=False,
            style=self.style,
            layout = self.auto_width_layout
        )

          
        self.view_attributes_button = widgets.Button(
            description='View',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            layout = self.auto_width_layout
            
        )
        
        
        self.apply_scale_button = widgets.Button(
            description='Apply',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            layout = self.auto_width_layout
            
        )
        
        self.view_correlation_button = widgets.Button(
            description='View',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='view correlations',
            layout = self.auto_width_layout
            
        )
        
        
        self.drop_features_button = widgets.Button(
            description='Drop',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Drop the selected features',
            layout = self.auto_width_layout
            
        )
        
        
        self.proxy_attributes_button = widgets.Button(
            description='Save',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Identify any possible proxy attributes',
            layout = self.auto_width_layout
            
        )
        
        self.ML_output_attributes_button = widgets.Button(
            description='Save',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Identify any possible ML generarted attributes',
            layout = self.auto_width_layout
            
        )
        
        self.save_file_button = widgets.Button(
            description='Save Transformed data',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Save the transformed data to csv',
            layout = self.auto_width_layout
            
        )
        
        
        
        
        self.progressBar = widgets.FloatProgress(
            value=0.0, 
            min=0.0, 
            max=1.0,
            layout = self.auto_width_layout,
        )
        
        
        self.viewProtectedOut  = widgets.Output(layout={})        

        
        #functions called on buttons clicked....
        self.remove_rows_cols_button.on_click(self.on_remove_rows_cols_button_clicked)
        self.apply_imputation_button.on_click(self.on_apply_imputation_button_clicked)
        self.view_attributes_button.on_click(self.on_view_attributes_button_clicked)
        self.merge_values_button.on_click(self.on_merge_button_clicked)
        self.save_description_button.on_click(self.on_save_description_button_clicked)
        self.change_column_type_button.on_click(self.on_change_column_type_button_clicked)
        self.label_encode_button.on_click(self.on_label_encode_button_clicked)
        self.one_hot_encode_button.on_click(self.on_one_hot_encode_button_clicked)
        self.apply_scale_button.on_click(self.on_apply_scale_button_clicked)
        self.view_correlation_button.on_click(self.on_view_correlation_button_clicked)
        self.drop_features_button.on_click(self.on_drop_features_button_clicked)
        self.proxy_attributes_button.on_click(self.on_proxy_attributes_button_clicked)
        self.ML_output_attributes_button.on_click(self.on_ML_output_attributes_button_clicked)
        self.save_file_button.on_click(self.on_save_file_button_clicked)
        
        

        #interactive outputs, the functions will be called with any activity on the controls specified
        #the input to the function will also be the .value of the controls that cause invocation
        self.uploadFileOutput = widgets.interactive_output(self.uploadFileFunction, {
                                'fileUploader_value': self.fileUploader}) 
        self.uploadFileOutput.layout.height = '250px'
        
        self.dropOutput = widgets.interactive_output(self.dropImputeFunction, {
                        'missingDataColumnsThresholdSlider_value': self.missingDataColumnsThresholdSlider,
                        'missingDataRowsThresholdSlider_value': self.missingDataRowsThresholdSlider,
                        'selectColumnWithMissingData_value': self.selectColumnWithMissingData})
        self.imputeOutput = widgets.Output(layout={})
        self.selectLabelYOutput = widgets.interactive_output(self.selectLabelYFunction, {
                                'selectYFeature_values': self.selectYFeature})
        self.selectLabelYOutput.layout.height = '120px'
        self.selectLabelXOutput = widgets.interactive_output(self.selectLabelXFunction, {
                                'selectXFeatures_values': self.selectXFeatures,
                                'selectProtectedAttributes_values': self.selectProtectedAttributes})
        self.selectLabelXOutput.layout.height = '150px'
        self.selecImpactOutput = widgets.interactive_output(self.selectImpactFunction, {
                                'selectImpact_values': self.selectImpact})
        
        self.workInProgressOut = widgets.Output(layout={})
        self.mergeOut = widgets.interactive_output(self.merge_Feature, {
                        'theSelectedInputFeaturesDropDown_value': self.theSelectedInputFeaturesDropDown,})
        self.saveDescriptionOut  = widgets.Output(layout={})
        

        
        self.theSelectedInputFeaturesChoiceOut = widgets.interactive_output(self.selected_features_choice, {
                        'theSelectedInputFeaturesChoice_value': self.theSelectedInputFeaturesChoice,})
        self.theSelectedInputFeaturesChoiceOut.layout.height = '100px'
        
        self.theSelectedInputFeaturesDropDownOut = widgets.interactive_output(self.selected_features_drop_down, {
                        'theSelectedInputFeaturesDropDown_value': self.theSelectedInputFeaturesDropDown,})
        self.theSelectedInputFeaturesDropDownOut.layout.height = '50px'
        
        self.LEOut = widgets.Output(layout={})

        self.HotEncodeOut = widgets.Output(layout={})

        self.scaleNormaliseOut = widgets.interactive_output(self.on_select_scale_type, {
                        'selectScaleType_value': self.selectScaleType,})
        self.changeDataTypeOut = widgets.Output(layout={})

        self.correlationsOut = widgets.Output(layout={})
        self.correlationsGraphOut = widgets.Output(layout={})
        self.dropColOut = widgets.Output(layout={})           

        self.protectedOut = widgets.interactive_output(self.withOrWithoutProtected, {
                            'selectProtected_value': self.selectProtected})
       
    
        #Create and display a FileChooser widget with path = current path
        self.fc = FileChooser()
        self.fc.show_hidden = True
        # Show or hide folder icons
        self.fc.use_dir_icons = True
        self.fc.default_filename = 'transformed_data.csv'
       




            
    #################################################################################################
    #  
    # 
    #################################################################################################     
    def uploadFileFunction(self, fileUploader_value):
        try:
            for key in fileUploader_value:
                text = "File uploaded: " + fileUploader_value[key]['metadata']['name']
                self.df_url = fileUploader_value[key]['metadata']['name']
                self.helper_methods.display_html (text, self.text_color, "h4")
                try:
                    csvInBytes = fileUploader_value[key]['content']
                    s = str(csvInBytes, 'utf-8')
                    data = StringIO(s)
                    self.df = pd.read_csv(data)
                    if self.df.columns[0] ==  "Unnamed: 0":
                        self.df = self.df.drop(["Unnamed: 0"], axis=1)
                    del csvInBytes
                    del data
                    del s
                    self.helper_methods.display_html ("Sample of data",self.text_color, "h3")
                    display (self.df.head(5))
                    self.selectColumnWithMissingData.options = self.df.columns[self.df.isnull().any()]
                    
                        
                except BaseException as error:      
                    text = "Error reading file - upload new file"
                    self.helper_methods.display_html (text, "red")
                    display('An exception occurred: {}'.format(error))
        except BaseException as error:
            display("Error uploading file - upload again")
            display('An exception occurred: {}'.format(error))
            
        self.selectXFeatures.options = self.df.columns.values
        self.selectProtectedAttributes.options = self.df.columns.values
        self.selectYFeature.options = self.df.columns.values
        gc.collect()

    #################################################################################################
    #  
    # 
    ################################################################################################# 
    def on_remove_rows_cols_button_clicked(self, b):
        thresholdCol = self.missingDataColumnsThresholdSlider.value
        thresholdRow = self.missingDataRowsThresholdSlider.value
        #Dropping columns with missing value rate higher than threshold
        self.df = self.df[self.df.columns[self.df.isnull().mean() < thresholdCol]]
            
        #Dropping rows with missing value rate higher than threshold
        self.df = self.df.loc[self.df.isnull().mean(axis=1) < thresholdRow]
        self.on_display_missing_data_info()

    
    #################################################################################################
    #  
    # 
    ################################################################################################# 
    def on_apply_imputation_button_clicked(self, b):
        #Filling all missing values with 0
        self.df[self.selectColumnWithMissingData.value].fillna(self.selectImputeMethod.value, inplace=True)
        self.on_display_missing_data_info()
        
    #################################################################################################
    #  
    # 
    ################################################################################################# 
    def on_display_missing_data_info(self):
        with self.dropOutput:
            clear_output(wait=True)
            thresholdCol = self.missingDataColumnsThresholdSlider.value
            thresholdRow = self.missingDataRowsThresholdSlider.value

            null_columns=self.df.columns[self.df.isnull().any()]
            all_columns=self.df.columns.values
            if len(null_columns) != 0:
                #How many columns have missing value rate higher than threshold
                text = "Number of columns with missing data ratio above " + str(thresholdCol) + " percent"
                num = self.df[self.df.columns[self.df.isnull().mean() > thresholdCol]].any(axis=0).sum()
                text = text + " is: " + str(num)
                self.helper_methods.display_html (text, self.text_color, "p")
            
                #How many rows with missing value rate higher than threshold
                text = "Number of rows with missing data ratio above "  + str(thresholdRow) + " percent"
                num = self.df.loc[self.df.isnull().mean(axis=1) > thresholdRow].any(axis=1).sum() 
                text = text + " is: " + str(num)
                self.helper_methods.display_html (text, self.text_color, "p")
            
                num_rows = self.df.shape[0] 
                text = "Total Number of Rows: "+ str(num_rows)
                self.helper_methods.display_html (text, self.text_color, "p")
            
                num_cols = self.df.shape[1] 
                text = "Total Number of Columns: "+ str(num_cols)
                self.helper_methods.display_html (text, self.text_color, "p")
            
                num_rows = self.df.isnull().any(axis=1).sum() 
                text = "Number of Rows with missing data: "+ str(num_rows)
                self.helper_methods.display_html (text, self.text_color, "p")
                #display(self.df[self.df.isnull().any(axis=1)][all_columns].head())
            
                self.selectColumnWithMissingData.options = self.df[null_columns].isnull().sum().index
                num_cols = len(self.selectColumnWithMissingData.options)
                text = "Number of Columns with missing data:" + str(num_cols)
                self.helper_methods.display_html (text, self.text_color, "p")
            else:
                num_rows = self.df.shape[0] 
                text = "Total Number of Rows: "+ str(num_rows)
                self.helper_methods.display_html (text, self.text_color, "p")
            
                num_cols = self.df.shape[1] 
                text = "Total Number of Columns: "+ str(num_cols)
                self.helper_methods.display_html (text, self.text_color, "p")
                
                text = "No missing data detected"
                self.helper_methods.display_html (text, "green", "p")
            return null_columns
        
        
    #################################################################################################
    #  
    # 
    #################################################################################################    
    def dropImputeFunction (self, missingDataColumnsThresholdSlider_value, missingDataRowsThresholdSlider_value, selectColumnWithMissingData_value):
        if not self.df.empty:
            null_columns = self.on_display_missing_data_info()
            
            numeric = self.df[null_columns].median().index
            with self.imputeOutput:
                clear_output(wait=True)
                optionsDic = {}
                if len(null_columns) != 0:
                    if selectColumnWithMissingData_value in numeric:
                    
                        median = self.df[selectColumnWithMissingData_value].median()
                        text = "The Median value for column: "+ selectColumnWithMissingData_value + " is " + str(median)
                        optionsDic["Median"] = median
                        optionsDic["Zero"] = 0
                        self.selectImputeMethod.options = optionsDic
                        self.helper_methods.display_html (text, self.text_color, "p")
                    
                    if not selectColumnWithMissingData_value in numeric:
                        display(self.df[selectColumnWithMissingData_value].value_counts().to_frame())
                        most_frequent = self.df[selectColumnWithMissingData_value].value_counts().idxmax()                
                        text = "Most frequent categorical value for " + selectColumnWithMissingData_value + " is " + str(most_frequent)
                        optionsDic["Most Frequent"] = str(most_frequent)
                        optionsDic["Other"] = "Other"
                        self.selectImputeMethod.options = optionsDic
                        self.helper_methods.display_html (text, self.text_color, "p")
                
                    num_missing = self.df[selectColumnWithMissingData_value].isnull().sum()
                    text = "Instances of missing data for selected column is " + str(num_missing)
                    self.helper_methods.display_html (text, self.text_color, "p")
                else:
                    text = "No missing data detected"
                    self.helper_methods.display_html (text, "green", "p")
    
                    
    #################################################################################################
    #  
    # 
    #################################################################################################                 
    def selectLabelYFunction(self, selectYFeature_values):
        
        if not self.df.empty:
           
            #Once we select a label, remove it from possible input features dropdown
            try:
                #store the entire label column in its own dataframe
                self.y_label_df = self.df[selectYFeature_values].copy().to_frame()
           
                listx = list(self.df.columns.values) 
                #use different ways to remove an item of the list
                listx.remove(selectYFeature_values) 
                #converting the tuple to list
                tuplex = tuple(listx)
                self.selectXFeatures.options =tuplex
                self.selectProtectedAttributes.options =tuplex

            
            except:
                self.selectXFeatures.options = self.df.columns.values
                self.selectProtectedAttributes.options = self.df.columns.values
                
        
            if selectYFeature_values == None:
                self.helper_methods.display_html ("None Selected", self.text_color, "h5")
            else:
                self.helper_methods.display_html (str(selectYFeature_values), self.text_color, "h5")
        
        
            if selectYFeature_values != None:
                colType = self.df[selectYFeature_values].dtype
                if (colType != str and colType != object):
                    if len(self.df[self.selectYFeature.value].unique()) == 2:
                        result =  all(elem in [1,0]  for elem in list(self.df[self.selectYFeature.value].unique()))
                        if result == True :
                            print ("The data to predict is binary. For ranking by default low vs high will depend on the mean ")
                            yMin = round(
                            self.df[selectYFeature_values].min(), 3)
                            yMean = round(
                            self.df[selectYFeature_values].mean(), 3)
                            yMax = round(
                            self.df[selectYFeature_values].max(), 3)
                        else:
                            print ("There are two output classes, they should be converted to binary before using the tool")
                            return
                        self.Y_BINARY = True
                        self.Y_CONTINUOUS  = False
                    else:
                        print ("The data to predict is continuous, For ranking by default low vs high will depend on the mean ")
                        self.Y_BINARY = False
                        self.Y_CONTINUOUS  = True
                        yMin = round(
                        self.df[selectYFeature_values].min(), 3)
                        yMean = round(
                        self.df[selectYFeature_values].mean(), 3)
                        yMax = round(
                        self.df[selectYFeature_values].max(), 3)
                
                    text = "Low Range(Minimum to Mean value): = " + str(yMin) + " to " + str(yMean) + "<br>"
                    text = text + "High Range(Mean to Maximum value): = " + str(yMean) + " to " + str(yMax) + "<br>"
                    
                    self.helper_methods.display_html (text, self.text_color, "p")
                 
                if (colType == str or colType == object):
                    self.helper_methods.display_html ("!The selected output label is not numeric", "orange", "p")
    
            
            
    #################################################################################################
    #  
    # 
    #################################################################################################           
    def selectImpactFunction(self, selectImpact_values):
        
        #Default value is High
        self.helper_methods.display_html ("Your choice indicates that the higher the predicted score the better the outcome for the individual", self.text_color,  "p")

        if not self.y_label_df.empty and not self.protected_feature_set_df.empty:
            if selectImpact_values == 1:
                self.HIGH_RANGE_POSITIVE = True
                self.helper_methods.display_html ("Your choice indicates that the higher the predicted score the better the outcome for the individual", self.text_color,  "p")
            else:
                self.HIGH_RANGE_POSITIVE = False
                self.helper_methods.display_html ("Your choice indicates that the lower the predicted score the better the outcome for the individual",  self.text_color, "p")

            with self.viewProtectedOut:
                clear_output(wait=True)
    
    
    #################################################################################################
    #  
    # 
    ################################################################################################# 
    def selectLabelXFunction(self, selectXFeatures_values, selectProtectedAttributes_values):
        if not self.df.empty:
            self.non_protected_feature_set_df = self.df
            #drop list contains all the Features not yet selected as X value.
            drop_list = set(selectXFeatures_values).symmetric_difference(
                self.selectXFeatures.options)
            self.helper_methods.display_html ("- Possible input features(X):","black", "h4")
            if len(selectXFeatures_values) == 0:
                self.helper_methods.display_html ("None Selected", self.text_color, "h4")
            else:
                self.helper_methods.display_html (str(selectXFeatures_values), self.text_color, "h4")
        
            self.helper_methods.display_html ("- Feature identifying membership of a protected group:", "black", "h4")
            if len(selectProtectedAttributes_values ) == 0:
                self.helper_methods.display_html ("None Selected", self.text_color, "h5")
            else:
                self.helper_methods.display_html (str(selectProtectedAttributes_values ), self.text_color, "h4")
            
            for c in drop_list:
                try:
                    self.non_protected_feature_set_df = self.df.drop(columns = c, axis=1)
                except:
                    print(c + "not a selected feature")
            self.non_protected_feature_set_df = self.non_protected_feature_set_df.drop(columns = self.selectYFeature.value, axis=1) 
        
            
            if len(selectProtectedAttributes_values) != 0:
                for c in list(selectProtectedAttributes_values):
                    try:
                        self.non_protected_feature_set_df = self.non_protected_feature_set_df.drop(columns = c, axis=1) 
                    except:
                        print(c + "not a selected feature")
                self.protected_feature_set_df = self.df[list(selectProtectedAttributes_values)].copy()

            #get a list of protected and non-protected based on the data frame 
            self.get_all_engineered_features()[2]#we only want 3d output of funct
            
    #################################################################################################
    #  get_dataframe()[0] to only return df
    # get_dataframe()[1] to only return column values
    ################################################################################################# 
    def get_dataframe(self, include_label_y, include_protected):
        df =  self.non_protected_feature_set_df
        if include_protected == True:
            df = pd.concat([df, self.protected_feature_set_df], axis=1)   
        if include_label_y == True:
            df = pd.concat([df, self.y_label_df], axis=1)
        return df, df.columns.values
     
    #################################################################################################
    #  
    # 
    #################################################################################################        
    def work_in_progress(self, progress):
        total = 100
        for i in range(total):
            time.sleep(0.2)
            progress.value = float(i+1)/total
            
    

    #################################################################################################
    #  
    # 
    #################################################################################################
    def on_view_attributes_button_clicked (self, b):
        if not self.y_label_df.empty and not self.protected_feature_set_df.empty:
            feature_set_with_y = self.get_dataframe(include_label_y = True, include_protected = True)[0]
            try:
                with self.viewProtectedOut:
                    clear_output(wait=True)
                    self.helper_methods.view_protected(list (self.selectProtectedAttributes.value), 
                                                       self.selectYFeature.value, feature_set_with_y, 
                                                       _w=600, _h=600, 
                                                       y_high_positive = self.HIGH_RANGE_POSITIVE,
                                                      output_type_binary = self.Y_BINARY)
                    
            except:
                print("an error occured viewing protected group")
        
 
    #################################################################################################
    #  
    # 
    #################################################################################################
    def get_all_engineered_features(self): 
        protected = []
        non_protected = []
        all_features = []
        without_engineered_original = []
        for feature in self.protected_feature_set_df.columns.values:
            protected.append(feature)
            all_features.append(feature)
            if not feature.endswith("_bm") and not feature.endswith("_benc"):
                without_engineered_original.append(feature)
            
        for feature in self.non_protected_feature_set_df.columns.values:
            non_protected.append(feature)
            all_features.append(feature)
            if not feature.endswith("_bm") and not feature.endswith("_benc"):
                without_engineered_original.append(feature)

        self.theSelectedInputFeaturesDropDown.options = without_engineered_original 
        self.theSelectedInputFeaturesChoice.options = without_engineered_original

        return protected, non_protected, all_features
        
    #################################################################################################
    #  
    # 
    #################################################################################################
    def merge_Feature(self, theSelectedInputFeaturesDropDown_value):
        if not self.non_protected_feature_set_df.empty:

            if not self.y_label_df.empty and not self.non_protected_feature_set_df.empty:
                try:

                    protected, non_protected, all_features = self.get_all_engineered_features()
           
                    p = set(protected)# for fast reference operation
                    
                    
                    if theSelectedInputFeaturesDropDown_value in p:# if protected
                        self.choosenFeatureValues.options = self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].unique()
                        colType = self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].dtype
                    else: 
                        self.choosenFeatureValues.options = self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].unique()
                        colType = self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].dtype
                    if len(self.choosenFeatureValues.value) == 1:
                        self.merge_values_button.description = "Change value"
                    else:
                        self.merge_values_button.description = "Merge values"
                    with self.mergeOut:
                        clear_output(wait=True)
                        if theSelectedInputFeaturesDropDown_value in p:
                            text = "The selected Feature is one of your protected Features."
                            self.helper_methods.display_html (text, self.text_color, "p")
                            
                    with self.changeDataTypeOut:
                        clear_output(wait=True)
                        text = "Feature data type: " + str(colType)
                        self.helper_methods.display_html (text, self.text_color, "h4")
                        
                except Exception as e:
                    
                    self.helper_methods.display_html ("**Protected attribut merge not yet generated**", self.text_color, "h4")
                    print(e)
    
    
    
    #################################################################################################
    #  
    # 
    #################################################################################################
    def on_merge_button_clicked(self, b):
        with self.mergeOut:
            clear_output(wait=True)
            protected = []
            for feature in self.protected_feature_set_df.columns.values:
                protected.append(feature)
            p = set(protected)# for fast reference operation, to see if we are renaming a protected Feature.
            
            colTypeCast = type(self.choosenFeatureValues.value[0])
            
            if self.theSelectedInputFeaturesDropDown.value not in self.renamed:
                self.renamed[self.theSelectedInputFeaturesDropDown.value] = {}
                
            for value in self.choosenFeatureValues.value:
                newValue = colTypeCast(self.newNameForValue.value)
                self.renamed[self.theSelectedInputFeaturesDropDown.value][value] = newValue
                newValue = colTypeCast(self.newNameForValue.value)
                
                if self.theSelectedInputFeaturesDropDown.value in p:# if protected
                    if not self.theSelectedInputFeaturesDropDown.value+"_bm" in self.protected_feature_set_df.columns:
                        self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value+"_bm"] = self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value]
                    self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].replace(
                        [value], newValue, inplace=True)
                else:
                    if not self.theSelectedInputFeaturesDropDown.value+"_bm" in self.non_protected_feature_set_df.columns:
                        self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value+"_bm"] = self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value]
                    self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].replace(
                        [value], newValue, inplace=True) #######
     
            ####show stats after rename
            display("Statistics after Merge:")
            if self.theSelectedInputFeaturesDropDown.value in p:# if protected
                display(self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].value_counts())
                colType = self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].dtype
                self.choosenFeatureValues.options = self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].unique()
           
            else:
                display(self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].value_counts())
                colType = self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].dtype
                self.choosenFeatureValues.options = self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].unique()
           
            if colType == str or colType == object :
                colType = str
                self.choosenFeatureValues.value = tuple([colType(newValue)])
                
            if colType == int or colType == float:
                self.choosenFeatureValues.value = tuple([newValue,])
                
            self.newNameForValue.value = "" #Reset the new name text box
                           
                        
    #################################################################################################
    #  
    # 
    #################################################################################################        
    def on_save_description_button_clicked(self, b):
        with self.saveDescriptionOut:
            clear_output(wait=True)
            
            if self.theSelectedInputFeaturesDropDown.value not in self.referenceNames:
                    self.referenceNames[self.theSelectedInputFeaturesDropDown.value] = {}
            #print ("options selected", self.theSelectedInputFeaturesDropDown.value)
            for value in self.choosenFeatureValues.value:
            
                text = "Saving description of: " + str(value) + " as "+ str(self.descriptionText.value) 
                display(text)
                self.referenceNames[self.theSelectedInputFeaturesDropDown.value][value] = self.descriptionText.value
                print(self.referenceNames)
        self.descriptionText.value = ""
    
   
    #################################################################################################
    #  displayed when values selected for hot-encoding
    # 
    #################################################################################################
    def selected_features_drop_down(self, theSelectedInputFeaturesDropDown_value):
        if not self.df.empty:
            with self.theSelectedInputFeaturesDropDownOut:
                clear_output(wait=True)
                protected, non_protected, all_features = self.get_all_engineered_features()
                selected = theSelectedInputFeaturesDropDown_value
                text = selected
                if selected in protected:# if protected
                    text = text + ": values " + str(self.protected_feature_set_df[selected].unique())
                    self.helper_methods.display_html (text, self.text_color, "h4")
                else: 
                    text = text + ": values " + str(self.non_protected_feature_set_df[selected].unique())
                    self.helper_methods.display_html (text, self.text_color, "h4")
                    
                        
    #################################################################################################
    # called when the theSelectedInputFeaturesChoice_value changes(e.g when selected for one-hot encode, 
    # scaling, etc
    #################################################################################################
    def selected_features_choice(self, theSelectedInputFeaturesChoice_value):
        if not self.df.empty:
            with self.theSelectedInputFeaturesChoiceOut:
                clear_output(wait=True)
                protected, non_protected, all_features = self.get_all_engineered_features()
                #Show th values when one feature is selected, just for convenience before selecting
                if len (theSelectedInputFeaturesChoice_value) == 1:
                    for selected in theSelectedInputFeaturesChoice_value:
                        text = selected
                        if selected in protected:# if protected
                            text = text + ": values to encode " + str(self.protected_feature_set_df[selected].unique())
                            self.helper_methods.display_html (text, self.text_color, "h4")
                        else: 
                            text = text + ": values to encode " + str(self.non_protected_feature_set_df[selected].unique())
                            self.helper_methods.display_html (text, self.text_color, "h4")

    #################################################################################################
    #  
    # 
    #################################################################################################                           
    def on_label_encode_button_clicked(self, b):
         if not self.non_protected_feature_set_df.empty:
            try:
                protected, non_protected, all_features = self.get_all_engineered_features()
                p = set(protected)# for fast reference operation
                prot_list = []
                non_prot_list = []
                prot_dict = {}
                non_prot_dict = {}
                for selected in self.theSelectedInputFeaturesChoice.value:
                    if selected in p:# if protected
                        prot_list.append(selected)   
                    else: 
                        non_prot_list.append(selected)

                if len(prot_list) > 0:
                    prot_dict = self.helper_methods.label_encoding(prot_list, self.protected_feature_set_df)                  
                    self.encoding_dict.update(prot_dict)
                if len(non_prot_list) > 0:
                    non_prot_dict = self.helper_methods.label_encoding(non_prot_list, self.non_protected_feature_set_df)                  
                    self.encoding_dict.update(non_prot_dict)
                
                text = ""
                for v in prot_list:
                    text = text + v + "_benc" + ", "
                for v in non_prot_list:
                    text = text + v + "_benc" + ", "
                with self.LEOut:
                    clear_output(wait=True)
                    text = "Label Encoding applied, original column copied as  " +  text
                    self.helper_methods.display_html (text, self.text_color, "p")
                    self.helper_methods.display_html (str(self.encoding_dict), self.text_color, "p")
                    self.get_all_engineered_features()  #to refresh the dropdowns with the new cols      
                    
            except Exception as e:
                with self.LEOut:
                    clear_output(wait=True)
                    self.helper_methods.display_html ("**Error when label encoding**", self.text_color, "h4")
                    display(e)
            
        
    #################################################################################################
    #  
    # 
    ################################################################################################# 
    def on_one_hot_encode_button_clicked(self, b):
        """ One Hot Encoding is used to convert categorical data, or text data, into numbers, 
        which our predictive models can better understand. 
        label encoding (an alternative approach)might introduces a new problems e.g if
        we encoded a set of country names into numerical data. 
        This is actually categorical data and there is no relation, of any kind, between the rows.
        The problem here is, since there are different numbers in the same column, 
        the model will misunderstand the data to be in some kind of order, 0 < 1 < 2. 
        But this isn’t the case at all. To overcome this problem, we use One Hot Encoder.
        One hot encoding is a process by which categorical variables are converted into a form that could be
        provided to ML algorithms to do a better job in prediction. 
        The categorical values start from 0 goes all the way up to N-1 categories.
        What one hot encoding does is, it takes a column which has categorical data, 
        which has been label encoded, and then splits the column into multiple columns. 
        The numbers are replaced by 1s and 0s, depending on which column has what value. 
        We want 7 new columns one for each Race category in our dataset."""
        with self.HotEncodeOut:
            clear_output(wait=True)
            protected, non_protected, all_features = self.get_all_engineered_features()
            p = set(protected)# for fast reference operation
                
            selected = self.theSelectedInputFeaturesDropDown.value
            text = "Applying one hot encoding to feature "+ str(selected)
            self.helper_methods.display_html (text, self.text_color, "p")
            if selected in p:# if protected
                self.protected_feature_set_df[selected] = pd.Categorical(self.protected_feature_set_df[selected])
                dfDummies = pd.get_dummies(self.protected_feature_set_df[selected], prefix=selected)
                self.protected_feature_set_df = pd.concat([self.protected_feature_set_df, dfDummies], axis=1)
                self.helper_methods.display_html("Generated Features: "+  str(dfDummies.columns.values), self.text_color, "h5")
                self.protected_feature_set_df.rename(columns={selected: selected+'_oh_benc'}, inplace=True)
            else: 
                self.non_protected_feature_set_df[selected] = pd.Categorical(self.non_protected_feature_set_df[selected])
                dfDummies = pd.get_dummies(self.non_protected_feature_set_df[selected], prefix=selected)
                self.non_protected_feature_set_df = pd.concat([self.non_protected_feature_set_df, dfDummies], axis=1)
                self.helper_methods.display_html("Generated Features: "+ str(dfDummies.columns.values), self.text_color, "h5")
                self.non_protected_feature_set_df.rename(columns={selected: selected+'_oh_benc'}, inplace=True)
            text = "Original column saved as "+ str(selected)+'_oh_benc'
            self.helper_methods.display_html (text, self.text_color, "p")
            self.get_all_engineered_features()  #to refresh the dropdowns with the new cols

    #################################################################################################
    #  
    # 
    ################################################################################################# 
    def on_change_column_type_button_clicked(self, b):
        with self.changeDataTypeOut:
            clear_output(wait=True)
            protected = []
            for feature in self.protected_feature_set_df.columns.values:
                protected.append(feature)
            p = set(protected)# for fast reference operation, to see if we are renaming a protected Feature.
            
            colName = self.theSelectedInputFeaturesDropDown.value
            newColType = self.selectColumnType.value
            
            
            if self.theSelectedInputFeaturesDropDown.value in p:# if protected
                colType = self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].dtype
                text_orig = "Original Data Type:" + str(colType)
                self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value] = self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].astype(newColType) 
                colType = self.protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].dtype
            
            else:
                colType = self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].dtype
                text_orig = "Original Data Type:" + str(colType)
                self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value] = self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].astype(newColType) 
                colType = self.non_protected_feature_set_df[self.theSelectedInputFeaturesDropDown.value].dtype
            
            self.helper_methods.display_html (text_orig, self.text_color, "p")
            text_new = "New Data Type:" + str(colType)
            self.helper_methods.display_html (text_new, self.text_color, "p")
            #self.refresh_views("change_column_type")
    
    
    
    #################################################################################################
    #  Method to view the scale of the input features. on_apply_scale button calls the same method
    # but with apply set to True for the helper file
    #################################################################################################
    def on_select_scale_type(self, selectScaleType_value):
        if not self.df.empty:
            if selectScaleType_value != "SELECT":
                self.scale_features(_apply = False)     
            
    def on_apply_scale_button_clicked(self, b):
        if not self.df.empty:
            self.scale_features(_apply = True)  
            
    #################################################################################################
    #  HERE: Have to view when select value, and apply when click button(so need to modify this flow)
    # 
    #################################################################################################
    def scale_features(self, _apply = False):
        with self.scaleNormaliseOut:
            clear_output(wait=True)
            try:
                protected, non_protected, all_features = self.get_all_engineered_features()
                p = set(protected)# for fast reference operation
                prot_list = []
                non_prot_list = []
                all_list = []

                for selected in self.theSelectedInputFeaturesChoice.value:
                    if selected in p:# if protected
                        prot_list.append(selected) 
                        all_list.append(selected) 
                    else: 
                        non_prot_list.append(selected)
                        all_list.append(selected)
                
                returned_df = self.helper_methods.scale_normalise(all_list, 
                                                    self.get_dataframe( include_label_y = False, include_protected = True)[0],                      
                                                    w=20, h=20, 
                                                    apply = _apply, 
                                                    s_type = self.selectScaleType.value);
                
                if _apply == True:
                    for col in prot_list:
                        self.protected_feature_set_df[col] = returned_df[col];
                    for col in non_prot_list:
                        self.non_protected_feature_set_df[col] = returned_df[col];
                    self.helper_methods.display_html ("Operation has been applied", self.text_color, "p")

                else:
                    self.helper_methods.display_html ("View only, press 'Apply' to apply", self.text_color, "p")
                
                
                del returned_df
                gc.collect()
                            
            except Exception as e:
                    self.helper_methods.display_html ("**Error when applying scale, select only numeric features**", self.text_color, "h4")
                    print(e)
            
        
    
    #################################################################################################
    # view_correlations: visualize the relationship between the features and the response using scatterplots
    # we need to choose variables that we think will be good predictors for the dependent variable — that can
    # be done by checking the correlation(s) between variables, by plotting the data and searching visually
    # for relationship, by conducting preliminary research on what variables are good predictors of y
    #################################################################################################
    def on_view_correlation_button_clicked(self, b):
        if not self.df.empty:
            try:
                with self.correlationsGraphOut :
                    clear_output(wait=False)
                    feature_set_with_y = self.get_dataframe(include_label_y = True, include_protected = True)[0]
                    column_list = list(self.theSelectedInputFeaturesChoice.value)
                    column_list_with_Y = list(self.theSelectedInputFeaturesChoice.value)+[self.selectYFeature.value]
                    protected_column_list = self.protected_feature_set_df.columns
                    self.helper_methods.view_correlations(feature_set_with_y, 
                                                          column_list, 
                                                          column_list_with_Y, 
                                                          protected_column_list,
                                                         self.selectYFeature.value)
                    
                    
                    # display (dpUI.non_protected_feature_set_df.head(5))
                    df_cols = self.get_dataframe(include_label_y=True, include_protected=True)[1]
                    to_use = []
                    for f in df_cols:
                        if not f.endswith("_bm") and not f.endswith("_benc"):
                            to_use.append(f)
                    display(HTML('''<h3>Pandas profile report</h3>
                                <br><a href="https://github.com/pandas-profiling/pandas-profiling" target="_blank">See details</a>'''))
                    profile = pandas_profiling.ProfileReport((self.get_dataframe(include_label_y=True, include_protected=True)[0])[to_use])
                    profile.to_widgets()
                    
            except:
                print("No values selected yet")
                
                
    #################################################################################################
    # Drop any columns that will not be used as input feature 
    # 
    #################################################################################################          
    def on_drop_features_button_clicked(self, b):
        if not self.df.empty:
            columns_to_drop = self.theSelectedInputFeaturesChoice.value 
            protected, non_protected, all_features = self.get_all_engineered_features()
            p = set(protected)# for fast reference operation

            for selected in columns_to_drop:
                if selected in p:# if protected
                    self.protected_feature_set_df.drop(selected, axis=1, inplace=True) 
                else: 
                    self.non_protected_feature_set_df.drop(selected, axis=1, inplace=True)
            with self.dropColOut:
                clear_output(wait=False)
                self.helper_methods.display_html ("Dropped "+ str (columns_to_drop),self.text_color, "h5")
                
    
    
    def on_proxy_attributes_button_clicked(self, b):
        if not self.df.empty:
            self.proxyAttributes = self.theSelectedInputFeaturesChoice.value
            print ("possible proxy attributes:",self.proxyAttributes  )
        
        
    def on_ML_output_attributes_button_clicked(self,b):
        if not self.df.empty:
            self.MLAttributes = self.theSelectedInputFeaturesChoice.value
            print ("possibly generated by ML:", self.MLAttributes )
            
        
        
    #################################################################################################
    #  NOT IN USE
    # Return the data with or without the protected attributes for the purpose of training
    ################################################################################################# 
    def withOrWithoutProtected(self, selectProtected_value):
        try:

            print("Input Feature set to use:")
            if (selectProtected_value == 'Yes'):
                self.helper_methods.display_html (str(self.non_protected_feature_set_df.columns.values+
                                                     self.protected_feature_set_df.columns.values), self.text_color, "h5")
    
            
            if (selectProtected_value == 'No'):
                self.helper_methods.display_html (str(self.non_protected_feature_set_df.columns.values), self.text_color, "h5")

        except Exception as e:
            display(e)

    
    #################################################################################################
    #  
    # 
    #################################################################################################
    def on_save_file_button_clicked(self, b):
        #Combine protected/non protected and y and save to csv
        df = pd.concat([self.non_protected_feature_set_df,self.protected_feature_set_df, self.y_label_df], axis=1)
        path = self.fc.selected_path + "/" + self.fc.selected_filename
        print ("File saved to", path)
        df.to_csv(path, index=False)#columns=['name']
        del df
        
        class file_data():
            def _init(self):
                self.renamed
                self.referenceNames
                self.df_url
                self.encoding_dict
                self.proxyAttributes 
                self.MLAttributes
                self.protected_x = []
                self.non_protected_x = []
                self.y_value
                self.all_columns_in_x
                self.HIGH_RANGE_POSITIVE
                self.Y_BINARY = False
                self.Y_CONTINUOUS = False
                self.html = {}


        protected_x = []
        non_protected_x = []

        html_summary = {}#Modify the below so that we have a block of HTML for any feature that has been engineered

        all_columns = list (self.protected_feature_set_df.columns) + list (self.non_protected_feature_set_df.columns)
        for column in (all_columns):
            text = ""
            if column.endswith("_oh_benc"):
                column = column[:-8] 
            if not column.endswith ("_bm") and not column.endswith ("_benc"):    
                if column+"_bm" in all_columns:
                    text  = text + "Feature: '" + column + "' has had some values merged<br>"
                    text = text + str (self.renamed[column] )  + "<br>" 
                if column in self.referenceNames:
                    text  = text + "Feature: '" + column + "' has some descriptions saved<br>"
                    text = text + str (self.referenceNames[column]) + "<br>"
                if column+"_benc" in all_columns:
                    text  = text + "Feature: '"+ column+ "' has been Label encoded<br>"
                    text = text + str (self.encoding_dict[column])  + "<br>" 
                if column+"_oh_benc" in all_columns:
                    text  = text + "Feature: '"+ column + "' has been One-Hot encoded<br>"
                    for x in all_columns:
                        if x.startswith(column+"_") and x != column+"_oh_benc":
                            text = text + str (x) + "<br>"
            
                html_summary[column] = text
                
        for column in (all_columns):
            if not column.endswith ("_bm") and not column.endswith ("_benc"):
                if column in self.protected_feature_set_df.columns:
                    protected_x.append(column)
                else:
                    non_protected_x.append(column)
                
        save_info = file_data()
        save_info.renamed = self.renamed
        save_info.referenceNames = self.referenceNames
        save_info.df_url = self.df_url
        save_info.encoding_dict = self.encoding_dict
        save_info.proxyAttributes  = self.proxyAttributes
        save_info.MLAttributes = self.MLAttributes
        save_info.y_value = (list(self.y_label_df.columns))[0]
        save_info.HIGH_RANGE_POSITIVE = self.HIGH_RANGE_POSITIVE
        save_info.Y_BINARY = self.Y_BINARY
        save_info.Y_CONTINUOUS  = self.Y_CONTINUOUS
        save_info.protected_x = protected_x
        save_info.non_protected_x = non_protected_x
        save_info.html = html_summary
        save_info.all_columns_in_x = all_columns
        a, b, c  = self.get_all_engineered_features() #prot, non prot, all
        
        
        

    # Save the file
        path = self.fc.selected_path + "/" + self.fc.selected_filename + "_summary.pickle"
        print ("File saved to", path)
        dill.dump(save_info, file = open(path, "wb"))
        
    
    #################################################################################################
    #  
    # 
    #################################################################################################

    def get_feature_set(self):
        return self.non_protected_feature_set_df.columns.values

    def get_protected(self):
        '''Returns:
        - a list of protected Features
        - a dictionary of renamed groups in a protected Feature
        - a dictionary of descriptions for a protected Feature.'''
        return self.protected_feature_set_df.columns.values
        

    def get_X_y(self):
        return self.non_protected_feature_set_df, self.protected_feature_set_df, self.y_label_df
        
    
    def get_pre_process_info(self):
        return self.encoding_dict, self.renamed, self.referenceNames
    
    def high_positive(self):
            return self.HIGH_RANGE_POSITIVE
            
    

            
        
    def render(self):
        
        box_layout = widgets.Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='90%')
        
    
        
        space = widgets.Label('  ', layout=widgets.Layout(width='100%'))

        uploadHTML = widgets.HTML(
            "<h3><left>1. Upload the data file to be used to train/test/validate your ML model :</left></h3>")
        
        removeRowsColumnsHTML = widgets.HTML(
            "<h3><left>2. Review and handle missing data:</left></h3> ")
        
        selectYHTML = widgets.HTML(
            """<h3><left>3. Select the Label(Y):</left></h3> 
            <font style="font-family:sans-serif; font-size:15px;color:grey;">
            I.e. The score that you want your final model to predict, bias can 
            cause an algorithm to miss the relevant causal relations between the input 
            features and this score(target output).""")
        
        
        selectImpactHTML = widgets.HTML(
            """<h3><left>4. Select an Impact the prediction will have on an individual:</left></h3>""")
        
        
        rankingEffectHTML = widgets.HTML(
            "<h4><center>Impact of output value</center></h4>")
        
        
        selectXHTML = widgets.HTML(
            """
            <h3><left>5. Select possible Input features and Protected Features</left></h3>
            <font style="font-family:sans-serif; font-size:15px;color:grey;">
            The possible input features are the available and observed features that a decision could be
            based upon. These do not necessarally reflect the “true” features of a person which are often not observable.
            In reality there may not be enough observable, measurable features to make ranking decisios between 
            individuals, as all we have is what has been measure or observed<br>

            <b>Fairness and Disparate Treatment</b> — A decision making process suffers from disparate 
            treatment if the decision is even partly based on a protected Feature. It generally involves 
            an intent to discriminate, however in the case of ML this can occur when the model 
            detects a corrilation between the feature in question(such as gender, race, religion or age) 
            and the expected labeled output used during the training process. This type of relative 
            discrimination can be detected if a predicted outcome changes on changing the value of the
            protected Feature. 
            
            <br><b> Note:</b> Later you can choose to exclude these protected attributes during model 
            training, however it will be important to retain this information as it pertains to validation
            /test data in order to measure potential bias in the final model.</b>  
            """
        )
        
                   
        
        proxyFeaturesHTML = widgets.HTML(  """ 
            <h3><left>15. Select any features that might be proxy attributes for a protected group</left></h3>
        
            <font style="font-family:sans-serif; font-size:15px;color:grey;">
            <b>Fairness and Disparate Impact</b> — A decision making process suffers from disparate impact if 
            the outcome of a decision disproportionately hurts (or benefits) those within a group. No intent 
            is required. This type of relative discrimination can be detected if there is a difference in the
            fraction of positive (negative) outcomes for the different sensitive groups. Discrimination can
            happen for example through other correlated features such as zip codes, or any input feature that is
            itself generated by an algorithm (for example credit scores).
            <br><b>Note:</b> Disparate impact is often referred to as unintentional discrimination,
            whereas disparate treatment is intentional. 
            """)

        
        MLOutputFeaturesHTML =  widgets.HTML("""
            <h3><left>16. Select any features that might themselves have been generated from a 
            ML algorithm</left></h3>
            <font style="font-family:sans-serif; font-size:15px;color:grey;">    
        """)
        
        
        ProtectedAttributesHTML = widgets.HTML(
            """<h3><left>6. Visualise protected attributes:</left></h3>
            <font style="font-family:sans-serif; font-size:15px;color:grey;">
             - View the representation of protected groups in the dataset.
            <br> - View the distribution of protected groups in the dataset.
            <br> - View the proportion of positive outcomes in the data set per group.
            <br>This analysis can be used to determine if there is an imbalanced representation of protected 
            groups in the dataset.We have not yet defined a bias metric therefor we refer here only to
            balance of data as opposed to any bias this may cause.
            """)

        mergeFeaturesHTML = widgets.HTML(
            """<h3><left>7. Merge Categorical feature levels(unique values)). </left></h3> 
            <font style="font-family:sans-serif; font-size:15px;color:grey;">
            This functionality can be used to reduce the number of levels(unique values) per categorical 
            feature by merging levels. For example you could merge "race" levels to form {White, Non-White}
            or gender to {Male, Non-Male}. The main motivation for merging levels(unique values) is to make 
            the model more robust and to prevent overfitting, which may occur if one level is
            disproportionately represented in the training data, the model will not understand an 
            underrepresente level very well (nor will it generalize). One option is to make an explicit 
            "other" category for values. There may however be a cost to performance as every time you merge
            levels, you sacrifice information and make your data more regularized. 
            <p style="color:green">Note: This functionality can also be use to rename a categorical
            feature level.</p>""")
        
        storeReferenceForFeaturesHTML = widgets.HTML(
            """<h3><left>8. Store description of feature value(s)</left></h3> 
            <font style="font-family:sans-serif; font-size:15px;color:grey;">
            The reason we might want to do this is so that we can review the data and model for bias 
            without applying any personal bias during the process. 
            We can also do this to help with reverse one-hot-encode/label encode later. 
            This dictionary will be returned with the newly generated data file """)
        
        selectLEHTML = widgets.HTML(
            """<h3><left>9. Label-Encode <b>Ordinal</b> categorical features. </left></h3>
            <font style="font-family:sans-serif; font-size:15px;color:grey;">
            Categorical features are those that contain a finite number of label values rather than 
            numeric values, ordinal categories are those that have a natural ordered relationship between 
            each other which the model may be able to understand and harness. Label-Encoding can also 
            be applied to nominal categorical features if there are only two categories, 
            as this is equivalent to One-Hot-Encoding(with "first column drop" applied)""")
        
        selectHEHTML = widgets.HTML(
            """<h3><left>10. One-Hot-Encode <b>Nominal</b> categorical features </left></h3>
            <font style="font-family:sans-serif; font-size:15px;color:grey;">
            <b>Nominal</b> Categorical features are those that that have NO natural ordered relationship e.g 
            Gender: {Female, Male}, or Race: {Asian, Black, Hispanic, White}. 
            Keep in mind Multicollinearity and the "One-Hot-Encoding trap"(Dummy Variable Trap.) in the case of
            gender for example if there are only two genders defined, it would be enough to label encode to 1/0 
            (or 0/1) as this is the equivalent to one-hot encoding except using one input feature instead of 
            two, thus avoiding the "dummy variable trap, when using one-hot encoding, it is common to leave 
            out one of the columns because it can be inferred as being that which is absent 
            (i.e. if all of the other columns in the one-hot encoded dummy variables are 0, 
            then it must be the other category". This can be achieved in step 13 if required""")
        
    
        changeValueFeaturesHTML = widgets.HTML(
            """<h3><left>11. Change the data type of a feature </left></h3> 
            <font style="font-family:sans-serif; font-size:15px;color:grey;">
            e.g. Change an int to a string or a string to an int""")
        
        
        selectScaleHTML = widgets.HTML(
            """<h3><left>12. Scale/Normalise input features </left></h3>
            <font style="font-family:sans-serif; font-size:15px;color:grey;">
            Many machine learning algorithms work better when features are on a relatively
            similar scale and close to normally distributed. 
            MinMaxScaler, RobustScaler, StandardScaler, and Normalizer are scikit-learn methods to 
            preprocess data for machine learning.""")

        
        correlationsHTML = widgets.HTML(
            """<h3><left>13. Feature Correlation Visualisation</left></h3>
            <font style="font-family:sans-serif; font-size:15px;color:grey;">
            Discrimination/Bais canoccur through features that behave as proxies for a protected feature,
            for example through the correlation of features such as zip codes, or any input feature that is
            itself generated by an algorithm (for example credit scores).
            
            
            """)

        dropFeaturesHTML = widgets.HTML(
            "<h3><left>14. Drop additional Input Features</left></h3>")

        "The integer values have a natural ordered relationship between each other and machine learning algorithms may be able to understand and harness this relationship."
         

        selectUseProtected = widgets.HTML(
            "<h4><left>15. Include protected features in model training?</left></h4>")
        
        selectSaveHTML = widgets.HTML(
            "<h4><left>15. Save transformed data to file.</left></h4>")
        
        
        todo = """ <br>Fairness as a philosophy has no objective definition and therefor there is no consensus on a 
            mathematical formulation for fairness. 
            <br>Something we have to consider when training a Machine learning model which will make a
            decision that has a positive or negative conseqnence on a person or group is if the historic
            decisions (labels) would have differed, or if representation of a particular group in the data 
            would be different had discrimination not occured in the pas. Is a loss in <b>percieved accuracy</b> 
            an acceptable trade off in an attempt to correct historic discrimination.
            
            Essentially, certain weights become too large and the whole system becomes biased. 
            Keeping all weights (for example in a DNN) around the same variance range forces the 
            network to use all pathways and generalise (underfit) better.
            
            </font>"""
        
        
        self.selectYFeature.layout.grid_area = 'topl'
        self.selectXFeatures.layout.grid_area = 'topl' 
        self.selectProtectedAttributes.layout.grid_area = 'topr'
        
        
        self.theSelectedInputFeaturesDropDown.layout.grid_area = 'topl'
        self.choosenFeatureValues.layout.grid_area  = 'topr'
        self.newNameForValue.layout.grid_area = 'middle'
        self.merge_values_button.layout.grid_area = 'bottom'

        
        self.theSelectedInputFeaturesDropDown.layout.grid_area = 'topl'
        self.descriptionText.layout.grid_area = 'middle'
        self.save_description_button.layout.grid_area = 'bottom'
        
        
        self.missingDataColumnsThresholdSlider.layout.grid_area = 'topl'
        self.missingDataRowsThresholdSlider.layout.grid_area = 'topr'
        self.remove_rows_cols_button.layout.grid_area = 'middle'
        self.selectColumnWithMissingData.layout.grid_area = 'topl'
        self.apply_imputation_button.layout.grid_area = 'bottoml'
        self.selectImputeMethod.layout.grid_area = 'topr'
        
        
        self.selectColumnType.layout.grid_area = 'topr'
        self.change_column_type_button.layout.grid_area = 'middle'
        
        self.selectXFeatures.layout.grid_area = 'topl'
        self.theSelectedInputFeaturesChoice.layout.grid_area = 'topl'
        self.theSelectedInputFeaturesDropDown.layout.grid_area = 'topl'
        self.view_attributes_button.layout.grid_area = 'topl'  
        self.label_encode_button.layout.grid_area = "topr"
        self.one_hot_encode_button.layout.grid_area =  "topr"
        self.selectColumnType.layout.grid_area  = "topr"
        self.change_column_type_button.layout.grid_area = "middlel"
        self.selectScaleType.layout.grid_area = "topr"
        self.apply_scale_button.layout.grid_area = "middlel"
                               
        self.view_correlation_button.layout.grid_area = "middlel"
        self.progressBar.layout.grid_area = "middle"
       
#         self.results_accordion.layout.grid_area = "bottom"
        self.drop_features_button.layout.grid_area = "bottom"
    
        self.proxy_attributes_button.layout.grid_area  = 'middle'  
        self.ML_output_attributes_button.layout.grid_area  = 'middle' 
            
        
        display(widgets.VBox([
            uploadHTML,
            self.fileUploader,
            self.uploadFileOutput,
            
            ###########Missing Data#################
            space,
            removeRowsColumnsHTML,
            widgets.GridBox(children=[self.missingDataColumnsThresholdSlider,
                                     self.missingDataRowsThresholdSlider,
                                     self.remove_rows_cols_button,
                                     ],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto auto',
                        grid_template_columns='50% 50%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl topr"
                        "middle ."
                        ''')
                   ),
            self.dropOutput,
            space,
            widgets.GridBox(children=[
                                     self.selectColumnWithMissingData,
                                     self.selectImputeMethod,
                                     self.apply_imputation_button],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto auto',
                        grid_template_columns='50% 50%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl topr"
                        "bottoml ."
                        ''')
                   ),
            self.imputeOutput,
            
            
            ###########Select Label (Y)#################
            space,
            selectYHTML,
            widgets.GridBox(children=[self.selectYFeature],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto',
                        grid_template_columns='50% 50%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl ."
                        ''')
                   ),        
            self.selectLabelYOutput,
            
            ###########Select Impact of Score#################
            space,
            selectImpactHTML,
            self.selectImpact,
            self.selecImpactOutput,
            
            
            ############Select Protected Attribute################
            space,
            selectXHTML,
            widgets.GridBox(children=[self.selectXFeatures, self.selectProtectedAttributes],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto',
                        grid_template_columns='50% 50%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl topr"
                        ''')
                   ),   
            self.selectLabelXOutput,
            
            
            ############View protected attribute stats################
            space,
            ProtectedAttributesHTML,
            widgets.GridBox(children=[self.view_attributes_button],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto auto',
                        grid_template_columns='50% 50%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl topl"
                        ''')
                   ),   
            
            self.viewProtectedOut,
            
            
            
            ##########Merge Feature values##################
            space,
            mergeFeaturesHTML,
            widgets.GridBox(children=[self.theSelectedInputFeaturesDropDown, self.choosenFeatureValues, self.newNameForValue, self.merge_values_button],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto auto auto',
                        grid_template_columns='45% 45%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl topr"
                        "middle ."
                        "bottom ."
                        ''')
                   ),        
            self.mergeOut,
            
            
            ############Store Description################
            space,
            storeReferenceForFeaturesHTML,
            widgets.GridBox(children=[self.theSelectedInputFeaturesDropDown, self.choosenFeatureValues, self.descriptionText, self.save_description_button],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto auto auto',
                        grid_template_columns='45% 45%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl topr"
                        "middle ."
                        "bottom ."
                        ''')
                   ),   
            self.saveDescriptionOut,
            
            ###########Label Encode#################  
            space,
            selectLEHTML,
            widgets.GridBox(children=[self.theSelectedInputFeaturesChoice, self.label_encode_button],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto',
                        grid_template_columns='45% 45%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl topr"
                        ''')
                   ),
            self.theSelectedInputFeaturesChoiceOut,
            self.LEOut,
            
            
            ###########One-Hot-Encode#################
            space,
            selectHEHTML,
            widgets.GridBox(children=[self.theSelectedInputFeaturesDropDown, self.one_hot_encode_button],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto',
                        grid_template_columns='45% 45%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl topr"
                        ''')
                   ), 
            self.theSelectedInputFeaturesDropDownOut,
            self.HotEncodeOut,
            
    
            
            
            ###########Change Type#################
            space,
            changeValueFeaturesHTML,
            widgets.GridBox(children=[self.theSelectedInputFeaturesDropDown, self.selectColumnType, self.change_column_type_button],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto auto ',
                        grid_template_columns='45% 45%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl topr"
                        "middlel ."
                        ''')
                   ),    
            self.theSelectedInputFeaturesDropDownOut,
            self.changeDataTypeOut,
            
            
            ###########Scale/Normalise Features#################
            space,
            selectScaleHTML,
            widgets.GridBox(children=[self.theSelectedInputFeaturesChoice, self.selectScaleType, self.apply_scale_button],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto',
                        grid_template_columns='45% 45%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl topr"
                        "middlel ."
                        ''')
                   ),   
            self.theSelectedInputFeaturesChoiceOut,
            self.scaleNormaliseOut,
            
            ###########View Correlation#################
            space,
            correlationsHTML,
            # self.show_correlation_btn,
            widgets.GridBox(children=[self.theSelectedInputFeaturesChoice, self.view_correlation_button],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto, auto',
                        grid_template_columns='45% 45%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl ."
                        "middlel ."
                        ''')
                   ),    
            self.correlationsOut,
            self.correlationsGraphOut,
            
            
            ##########Drop Features##################
            space,
            dropFeaturesHTML,
            widgets.GridBox(children=[self.theSelectedInputFeaturesChoice,self.drop_features_button ],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto auto',
                        grid_template_columns='45% 45%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl ."
                        "bottom ."
                        ''')
                   ), 
            self.dropColOut,
            
           
            ##########Are you aware of possible proxy features##################
            space,
            proxyFeaturesHTML,
            widgets.GridBox(children=[self.theSelectedInputFeaturesChoice,self.proxy_attributes_button],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto',
                        grid_template_columns='45% 45%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl ."
                        "middle ."
                        ''')
                   ), 
           
            
            ##########Are any of the features an output from a ML algorithm##################
            space,
            MLOutputFeaturesHTML,
            widgets.GridBox(children=[self.theSelectedInputFeaturesChoice, self.ML_output_attributes_button ],
                        layout=Layout(
                        width='90%',
                        grid_template_rows='auto',
                        grid_template_columns='45% 45%',
                        border= '1px solid green',
                        padding = '10px',
                        grid_gap='10px 10px',
                        grid_template_areas='''
                        "topl ."
                        "middle ."
                        ''')
                   ), 
            
        ]))
        
        display (
            selectSaveHTML,
            self.fc,
            self.save_file_button)
        
        


In [4]:
dpUI = data_pre_process_UI()
dpUI.render()

HTML(value='<h4><left>15. Save transformed data to file.</left></h4>')

FileChooser(path='/Users/aideenf/Documents/GitHub/Thesis_Aideen/Main', filename='transformed_data.csv', show_h…

Button(description='Save Transformed data', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='Save th…

""


<Figure size 432x288 with 0 Axes>

File saved to /Users/aideenf/Documents/GitHub/Thesis_Aideen/Main/Data/transformed_data.csv
File saved to /Users/aideenf/Documents/GitHub/Thesis_Aideen/Main/Data/transformed_data.csv_summary.pickle
